In [148]:
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.optimize import broyden1

In [149]:
pcecc = pd.read_csv("PCECC96.csv")
W5000 = pd.read_csv("WILL5000INDFC.csv")
gdpdef = pd.read_csv("GDPDEF.csv")
TB3MS = pd.read_csv("TB3MS.csv")
l = [pcecc, gdpdef, W5000, TB3MS]
for i, ds in enumerate(l):
    ds['DATE'] = pd.to_datetime(ds['DATE'])
    ds.set_index('DATE', inplace = True)
    ds[ds.columns[0]] = pd.to_numeric(ds[ds.columns[0]], errors='coerce')
    if i > 1:
        ds = ds.resample('D').asfreq()
        ds.fillna(method='ffill', inplace=True)
        ds = ds.resample('Q').asfreq()
        ds.index = ds.index + pd.DateOffset(1)
    if i == 0:
        df = ds
    else:
        df = df.merge(ds, "outer", left_index=True, right_index=True) 
df.dropna(inplace = True)
df['W5000_real'] = df['W5000'] / (df['GDPDEF'] / 100)
df['W5000_return'] = df['W5000_real'] / df['W5000_real'].shift(1)
df.head()

print(df)

              PCECC96   GDPDEF   W5000  TB3MS  W5000_real  W5000_return
DATE                                                                   
2000-01-01   8520.710   77.396   44.67   5.20   57.716161           NaN
2000-04-01   8603.007   77.865   46.37   5.69   59.551788      1.031804
2000-07-01   8687.485   78.309   44.29   5.69   56.557995      0.949728
2000-10-01   8762.205   78.723   44.37   6.00   56.362181      0.996538
2001-01-01   8797.280   79.204   39.80   5.77   50.249987      0.891555
2001-04-01   8818.079   79.683   34.89   4.42   43.786002      0.871363
2001-07-01   8848.300   80.004   37.50   3.49   46.872656      1.070494
2001-10-01   8980.610   80.268   31.53   2.64   39.280909      0.838035
2002-01-01   9008.096   80.533   35.43   1.69   43.994387      1.119994
2002-04-01   9054.348   80.821   35.77   1.79   44.258299      1.005999
2002-07-01   9119.949   81.194   31.26   1.70   38.500382      0.869902
2002-10-01   9172.361   81.654   26.01   1.63   31.853920      0

In [151]:
class Prob1:
    def __init__(self, df, beta):
        self.df = df
        self.beta = beta
        self.gamma = 0
        
    def emp_analogs(self, gamma):
        du = self.df['PCECC96'] ** (-gamma)
        self.df['M'] = self.beta * (du / du.shift(1))
        M = self.df['M'][1:]
        R = self.df['W5000_return'][1:]
        cov = np.cov(M, R)[0,1]
        rv = (1 - cov - M.mean() * R.mean())
        return rv
    
    def find_gamma(self, guess):
        gamma = broyden1(self.emp_analogs, guess)
        self.gamma = gamma
        return gamma
        

problem1 = Prob1(df, .99)
problem1.emp_analogs(1.5)
problem1.find_gamma(1.5)        

array(0.48719535)

Seems like a reasonable parameter, according to this working paper (http://karlshell.com/wp-content/uploads/2015/09/WebPage.pdf) estimates normally range from 1 - 4 with a mean of 2 so .48, while a bit low is not beyond the realm of possibility. It is also hard to know because the data are likely compromised due to market fluctuations around the great recession. 

$$ 1+r_{f}=\frac{1}{E\left[m_{t+1} | \Omega_{t}\right]} $$

In [168]:
class Prob3:
    def __init__(self, df, beta):
        self.df = df
        self.beta = beta
        self.gamma = 0
        
    def risk_free_obj(self, gamma):
        du = self.df['PCECC96'] ** (-gamma)
        self.df['M'] = self.beta * (du / du.shift(1))
        M = self.df['M'][1:]
        rf = (self.df['TB3MS'][1:]) / 100
        rv = 1 + rf.mean() - 1/M.mean()
        # rv = np.sum(eq ** 2)
        return rv
    
    def find_gamma(self, guess):
        gamma = broyden1(self.risk_free_obj, guess)
        self.gamma = gamma
        return gamma



problem3 = Prob3(df, .99)
problem3.find_gamma(3.5)   

array(1.00475361)

This value is significantly higher than the last one was and is more reasonable as an estimate

In [178]:
class Prob4:
    def __init__(self, df):
        self.df = df
        self.beta = 0
        self.gamma = 0
        
    def both(self, v):
        gamma, beta = v
        rv = []
        du = self.df['PCECC96'] ** (-gamma)
        # Condition 1 
        self.df['M'] = beta * (du / du.shift(1))
        M = self.df['M'][1:]
        R = self.df['W5000_return'][1:]
        cov = np.cov(M, R)[0,1]
        rv.append(1 - cov - M.mean() * R.mean())
        #Condition 2
        rf = (self.df['TB3MS'][1:]) / 100
        rv.append(1 + rf.mean() - 1/M.mean())
        return rv
    
        
    def find_gamma(self, guess):
        rv = broyden1(self.both, guess)
        self.gamma = rv[0]
        self.beta = rv[1]
        return rv
    
problem4 = Prob4(df)
problem4.find_gamma((1, 1)) 

array([-59.46324968,   0.67836582])